# Movie Recomendation - Similarity Model

The purpose of this motel is to get recomendations of movies based on a selected movie and related features.

In [1]:
# from nltk.corpus import stopwords 
# from nltk.tokenize import word_tokenize
# def stop_words(x):
#     text = ' '.join(x)
#     stop_words = set(stopwords.words('english')) 
#     word_tokens = word_tokenize(text) 
#     text = [w for w in word_tokens if not w in stop_words] 
#     return text

In [2]:
import pandas as pd
import numpy as np

## Importing Database

In [3]:
data = pd.read_csv('../raw_data/treated-db/data.csv')

In [4]:
data.dropna(inplace=True)

In [5]:
data

,movieId,tmdbId,title,title_clean,genres,actors,directors,overview,keywords,popularity,vote_count,vote_average,runtime,release
0,1.0,862,Toy Story (1995),['toy story'],"['adventure', 'animation', 'children', 'comedy...","['tomhanks', 'timallen', 'donrickles']",['johnlasseter'],"led by woody, andy's toys live happily in his ...","['martialarts', 'jealousy', 'toy', 'friendship...",70.910,13173.0,7.9,81.0,1995-10-30
1,2.0,8844,Jumanji (1995),['jumanji'],"['adventure', 'children', 'fantasy']","['robinwilliams', 'jonathanhyde', 'kirstendunst']",['joejohnston'],when siblings judy and peter discover an encha...,"['giantinsect', 'boardgame', 'jungle', 'disapp...",11.919,7711.0,7.2,104.0,1995-12-15
2,3.0,15602,Grumpier Old Men (1995),['grumpier old men'],"['comedy', 'romance']","['waltermatthau', 'jacklemmon', 'ann-margret']",['howarddeutch'],a family wedding reignites the ancient feud be...,"['fishing', 'oldman', 'bestfriend', 'duringcre...",13.122,217.0,6.6,101.0,1995-12-22
3,4.0,31357,Waiting to Exhale (1995),['waiting to exhale'],"['comedy', 'drama', 'romance']","['whitneyhouston', 'angelabassett', 'lorettade...",['forestwhitaker'],"cheated on, mistreated and stepped on, the wom...","['basedonnovelorbook', 'interracialrelationshi...",5.123,79.0,6.2,127.0,1995-12-22
4,5.0,11862,Father of the Bride Part II (1995),['father of the bride part ii'],['comedy'],"['stevemartin', 'dianekeaton', 'martinshort']",['charlesshyer'],just when george banks has recovered from his ...,"['parentchildrelationship', 'baby', 'midlifecr...",15.142,446.0,6.2,106.0,1995-12-08
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23308,208843.0,553882,An Acceptable Loss (2019),['an acceptable loss'],"['drama', 'thriller']","['tikasumpter', 'jamieleecurtis', 'bentavassoli']",['joechappelle'],"former top u.s. security adviser elizabeth ""li...","['regret', 'politician', 'betrayal', 'threat',...",7.757,42.0,6.3,102.0,2018-10-13
23309,208863.0,31006,Homicide: The Movie (2000),['homicide: the movie'],"['crime', 'drama']","['danielbaldwin', 'nedbeatty', 'richardbelzer']",['jeandesegonzac'],retired police lieutenant al giardello is runn...,"['usa', 'baltimore']",3.374,12.0,6.9,89.0,2000-02-13
23310,208915.0,639832,Undercover Brother 2 (2019),['undercover brother 2'],"['action', 'comedy']","['michaeljaiwhite', 'barrybostwick', 'jeffdani...",['lesliesmall'],"sixteen years ago, undercover brother and his ...",['spy'],19.101,14.0,3.9,85.0,2019-11-05
23311,208917.0,186021,Caicedo (with Pole) (1894),['caicedo (with pole)'],['documentary'],['juana.caicedo'],['williamk.l.dickson'],king of the slack wire. his daring feats of ba...,"['silentfilm', 'vaudeville', 'acrobats']",1.591,21.0,5.8,1.0,1894-07-25


## Database setup

In [6]:
import ast
def api_to_dict(x):
    return ast.literal_eval(x)

In [7]:
df = data.copy()

In [8]:
df.head(1)

,movieId,tmdbId,title,title_clean,genres,actors,directors,overview,keywords,popularity,vote_count,vote_average,runtime,release
0,1.0,862,Toy Story (1995),['toy story'],"['adventure', 'animation', 'children', 'comedy...","['tomhanks', 'timallen', 'donrickles']",['johnlasseter'],"led by woody, andy's toys live happily in his ...","['martialarts', 'jealousy', 'toy', 'friendship...",70.91,13173.0,7.9,81.0,1995-10-30


In [9]:
import ast
df = data.copy()
df.drop(columns=['popularity','movieId','vote_count','vote_average','runtime','release'],inplace = True)

df['actors'] = df['actors'].apply(api_to_dict)
df['keywords'] = df['keywords'].apply(api_to_dict)
df['genres'] = df['genres'].apply(api_to_dict)

df.head(1)

,tmdbId,title,title_clean,genres,actors,directors,overview,keywords
0,862,Toy Story (1995),['toy story'],"[adventure, animation, children, comedy, fantasy]","[tomhanks, timallen, donrickles]",['johnlasseter'],"led by woody, andy's toys live happily in his ...","[martialarts, jealousy, toy, friendship, bully..."


### Overview setup

#### Remove ponctuation and stopwords

In [12]:
import nltk
import string 
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [13]:
def remove_punctuation(x):
    for i in string.punctuation:
        x = x.replace(i, '')
    return x

In [14]:
def stop_words(x):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(x)
    x = [w for w in word_tokens if not w in stop_words] 
  
    return x

In [15]:
df['title_clean'] = df['title_clean'].apply(remove_punctuation)
df['title_clean'] = df['title_clean'].apply(stop_words)
df['overview'] = df['overview'].apply(remove_punctuation)
df['overview'] = df['overview'].apply(stop_words)
df['directors'] = df['directors'].apply(remove_punctuation)
df['directors'] = df['directors'].apply(stop_words)

In [16]:
def to_string(x):
    x = ' '.join(x)
    return x

In [17]:
#Tudão Bolado
df['soup_1'] = (df['genres'] + df['actors'] + df['keywords'] + df['overview'] + df['directors'] + df['title_clean']).map(to_string)

#Sem title
df['soup_2'] = (df['genres'] + df['actors'] + df['keywords'] + df['overview'] + df['directors']).map(to_string)

#Sem overview e title
df['soup_3'] = (df['genres'] + df['actors'] + df['keywords'] + df['directors']).map(to_string)

#Sem Key words e overview
df['soup_4'] = (df['genres'] + df['actors'] + df['directors'] + df['title_clean']).map(to_string)

#Sem Key words, Title e overview
df['soup_5'] = (df['genres'] + df['actors']  + df['directors']).map(to_string)

#sem Directors e overview
df['soup_6'] = (df['genres'] + df['keywords'] + df['actors']).map(to_string)

#Genres e KW
df['soup_7'] = (df['genres'] + df['keywords']).map(to_string)

#Genres e Overview
df['soup_8'] = (df['genres'] + df['overview']).map(to_string)

In [18]:
df.head(2)

,tmdbId,title,title_clean,genres,actors,directors,overview,keywords,soup_1,soup_2,soup_3,soup_4,soup_5,soup_6,soup_7,soup_8
0,862,Toy Story (1995),"[toy, story]","[adventure, animation, children, comedy, fantasy]","[tomhanks, timallen, donrickles]",[johnlasseter],"[led, woody, andys, toys, live, happily, room,...","[martialarts, jealousy, toy, friendship, bully...",adventure animation children comedy fantasy to...,adventure animation children comedy fantasy to...,adventure animation children comedy fantasy to...,adventure animation children comedy fantasy to...,adventure animation children comedy fantasy to...,adventure animation children comedy fantasy ma...,adventure animation children comedy fantasy ma...,adventure animation children comedy fantasy le...
1,8844,Jumanji (1995),[jumanji],"[adventure, children, fantasy]","[robinwilliams, jonathanhyde, kirstendunst]",[joejohnston],"[siblings, judy, peter, discover, enchanted, b...","[giantinsect, boardgame, jungle, disappearance...",adventure children fantasy robinwilliams jonat...,adventure children fantasy robinwilliams jonat...,adventure children fantasy robinwilliams jonat...,adventure children fantasy robinwilliams jonat...,adventure children fantasy robinwilliams jonat...,adventure children fantasy giantinsect boardga...,adventure children fantasy giantinsect boardga...,adventure children fantasy siblings judy peter...


In [19]:
# Import CountVectorizer and create the count matrix
from sklearn.feature_extraction.text import CountVectorizer

count1 = CountVectorizer()
count_matrix1 = count1.fit_transform(df['soup_1'])

count2 = CountVectorizer()
count_matrix2 = count2.fit_transform(df['soup_2'])

count3 = CountVectorizer()
count_matrix3 = count3.fit_transform(df['soup_3'])

count4 = CountVectorizer()
count_matrix4 = count4.fit_transform(df['soup_4'])

count5 = CountVectorizer()
count_matrix5 = count5.fit_transform(df['soup_5'])

count6 = CountVectorizer()
count_matrix6 = count6.fit_transform(df['soup_6'])

count7 = CountVectorizer()
count_matrix7 = count7.fit_transform(df['soup_7'])

count8 = CountVectorizer()
count_matrix8 = count8.fit_transform(df['soup_8'])

In [20]:
# Compute the Cosine Similarity matrix based on the count_matrix
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim1 = cosine_similarity(count_matrix1, count_matrix1)
cosine_sim2 = cosine_similarity(count_matrix2, count_matrix2)
cosine_sim3 = cosine_similarity(count_matrix3, count_matrix3)
cosine_sim4 = cosine_similarity(count_matrix4, count_matrix4)
cosine_sim5 = cosine_similarity(count_matrix5, count_matrix5)
cosine_sim6 = cosine_similarity(count_matrix6, count_matrix6)
cosine_sim7 = cosine_similarity(count_matrix7, count_matrix7)
cosine_sim8 = cosine_similarity(count_matrix8, count_matrix8)

In [ ]:
df2 = df.copy()

In [ ]:
# Reset index of our main DataFrame and construct reverse mapping as before
df2 = df2.reset_index()
indices = pd.Series(df2.index, index=df2['title'])

In [ ]:
df2

In [ ]:
sdfgadfgadfg

In [ ]:
# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(title, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    idx = indices[title]

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort the movies based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores = sim_scores[1:11]

    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores]

    # Return the top 10 most similar movies
    return df2['title'].iloc[movie_indices]

In [ ]:
get_recommendations('Dark Knight Rises, The (2012)', cosine_sim)